In [6]:
import torch
import torchvision
import numpy as np
import os.path as osp
import os
import time
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image
import load_alov
import sys
sys.path.append("..")
import numpy as np
import detect
from darknet import Darknet
from utils import *

In [2]:
CUDA = torch.cuda.is_available()
CUDA

True

In [10]:
# GLOBALS
vot_path = "/data/vot2016/"

yolo_classes = ['person', 'bicycle', 'car', 'motorbike', 'aeroplane', 'bus', 'train', 'truck', 'boat', 'traffic light', 'fire hydrant', 'stop sign', 'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse', 'sheep', 'cow', 'elephant', 'bear', 'zebra', 'giraffe', 'backpack', 'umbrella', 'handbag', 'tie', 'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball', 'kite', 'baseball bat', 'baseball glove', 'skateboard', 'surfboard', 'tennis racket', 'bottle', 'wine glass', 'cup', 'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple', 'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza', 'donut', 'cake', 'chair', 'sofa', 'pottedplant', 'bed', 'diningtable', 'toilet', 'tvmonitor', 'laptop', 'mouse', 'remote', 'keyboard', 'cell phone', 'microwave', 'oven', 'toaster', 'sink', 'refrigerator', 'book', 'clock', 'vase', 'scissors', 'teddy bear', 'hair drier', 'toothbrush']
['blanket', 'bolt2', 'traffic', 'ball1', 'leaves', 'pedestrian1', 'singer3', 'helicopter', 'godfather', 'soccer1', 'book', 'tunnel', 'gymnastics1', 'basketball', 'birds1', 'car1', 'ball2', 'birds2', 'motocross1', 'bmx', 'gymnastics2', 'fish3', 'motocross2', 'soccer2', 'octopus', 'fernando', 'shaking', 'crossing', 'singer2', 'soldier', 'pedestrian2', 'nature', 'butterfly', 'road', 'iceskater2', 'sphere', 'fish2', 'graduate', 'marching', 'rabbit', 'car2', 'matrix', 'girl', 'tiger', 'gymnastics4', 'wiper', 'racing', 'dinosaur', 'bolt1', 'hand', 'handball1', 'bag', 'sheep', 'iceskater1', 'fish1', 'singer1', 'gymnastics3', 'fish4', 'glove', 'handball2']

vot_classes = [x for x in os.listdir(vot_path) if not x.endswith(".txt")]

print(vot_classes)

['soccer1', 'road', 'sphere', 'glove', 'nature', 'matrix', 'ball1', 'fish1', 'singer3', 'bmx', 'book', 'shaking', 'sheep', 'girl', 'gymnastics1', 'motocross1', 'fish2', 'fernando', 'traffic', 'butterfly', 'ball2', 'marching', 'pedestrian1', 'car1', 'rabbit', 'handball2', 'crossing', 'tunnel', 'bag', 'blanket', 'dinosaur', 'fish3', 'singer2', 'racing', 'pedestrian2', 'motocross2', 'gymnastics3', 'iceskater1', 'birds1', 'hand', 'bolt2', 'bolt1', 'godfather', 'octopus', 'soldier', 'graduate', 'birds2', 'wiper', 'leaves', 'gymnastics4', 'handball1', 'singer1', 'gymnastics2', 'tiger', 'helicopter', 'fish4', 'car2', 'iceskater2', 'soccer2', 'basketball']


In [7]:
cfgfile = "../cfg/yolo.cfg"
weightfile = "../yolo.weights"
namesfile = '../data/coco.names'
use_cuda=True
m = Darknet(cfgfile).cuda()
m.print_network()
m.load_weights(weightfile)

layer     filters    size              input                output
    0 conv     32  3 x 3 / 1   416 x 416 x   3   ->   416 x 416 x  32
    1 max          2 x 2 / 2   416 x 416 x  32   ->   208 x 208 x  32
    2 conv     64  3 x 3 / 1   208 x 208 x  32   ->   208 x 208 x  64
    3 max          2 x 2 / 2   208 x 208 x  64   ->   104 x 104 x  64
    4 conv    128  3 x 3 / 1   104 x 104 x  64   ->   104 x 104 x 128
    5 conv     64  1 x 1 / 1   104 x 104 x 128   ->   104 x 104 x  64
    6 conv    128  3 x 3 / 1   104 x 104 x  64   ->   104 x 104 x 128
    7 max          2 x 2 / 2   104 x 104 x 128   ->    52 x  52 x 128
    8 conv    256  3 x 3 / 1    52 x  52 x 128   ->    52 x  52 x 256
    9 conv    128  1 x 1 / 1    52 x  52 x 256   ->    52 x  52 x 128
   10 conv    256  3 x 3 / 1    52 x  52 x 128   ->    52 x  52 x 256
   11 max          2 x 2 / 2    52 x  52 x 256   ->    26 x  26 x 256
   12 conv    512  3 x 3 / 1    26 x  26 x 256   ->    26 x  26 x 512
   13 conv    256  1 x 

In [68]:
def iou(box_a, box_b):
    # IoU between two boxes
    box_a = box_a[0:4] if len(box_a) != 4 else box_a
    box_b = box_b[0:4] if len(box_b) != 4 else box_b

    x_a = np.max((box_a[0], box_b[0]))
    y_a = np.max((box_a[1], box_b[1]))
    x_b = np.min((box_a[2], box_b[2]))
    y_b = np.min((box_a[3], box_b[3]))

    inter_area = np.max((0., x_b - x_a + 1)) * np.max((0., y_b - y_a + 1))
    box_a_area = (box_a[2] - box_a[0] + 1) * (box_a[3] - box_a[1] + 1)
    box_b_area = (box_b[2] - box_b[0] + 1) * (box_b[3] - box_b[1] + 1)
    a = (box_a_area + box_b_area - inter_area)
    res = inter_area / a
    return res


def choose_best_by_emb(prev, candidates, thr=.3):
    best_dist = thr
    best_track_idx = 0
    prev_emb = np.array(prev[7:-2])
    
    for n, candidate in enumerate(candidates):
        cand_emb = np.array(candidate[7:-2])
        emb_dist = torch.mean((cand_emb[0] - prev_emb[0]) ** 2)
        if emb_dist < best_dist:
            best_dist = emb_dist
            best_track_idx = n
    
    if best_dist < thr:
        return candidates[best_track_idx]
    return prev

def best_iou(candidates, gt, width=None, height=None):
    best_iou = 0
    best_index = 0
    for i, candidate in enumerate(candidates):
        c = candidate
        if len(c) > 4:
            c = c[0:4]
        if width is not None:
            x1 = (c[0] - c[2] / 2.0) * width
            y1 = (c[1] - c[3] / 2.0) * height
            x2 = (c[0] + c[2] / 2.0) * width
            y2 = (c[1] + c[3] / 2.0) * height
            c = np.array([x1, y1, x2, y2])
        _iou = iou(c, gt)
        if _iou > best_iou:
            best_iou = _iou
            best_index = i
    
    return candidates[best_index] if best_iou > 0 else None

def file2tensor(file):
    with open(file, 'r') as f:
        lines = torch.tensor([[float(y) for y in x.split(',')] for x in f.readlines()])
        return lines
    
def read_gt(file, idx=None):
    tensor = file2tensor(file)
    if idx is not None:
        return tensor[idx]
    else:
        return tensor
    
def force_square(m):
    return [min(m[::2]), min(m[1::2]), max(m[::2]), max(m[1::2])]

In [ ]:
def plot_single(im, box=None, line="r-", resize=False, noshow=False):
    if not noshow:
        plt.imshow(im)

#     if pr_path is not None:
#         with open(pr_path, 'r') as f:
#             l = f.read().split("\n")[pr_idx]
#             if l == "":
#                 return
#             pr_bb = [float(x) for x in l.split(',')]

#         if (force_square):
#             X, Y = pr_bb[::2], pr_bb[1::2]
#             pr_bb = [min(X), min(Y), max(X),  max(Y)]
#             plt.plot([pr_bb[0], pr_bb[2], pr_bb[2], pr_bb[0], pr_bb[0], ],
#                      [pr_bb[1], pr_bb[1], pr_bb[3], pr_bb[3], pr_bb[1], ], 'r-', lw=2)
#         elif (len(pr_bb) == 8):
#             plt.plot(pr_bb[::2] + [pr_bb[0]],
#                      pr_bb[1::2] + [pr_bb[1]], 'r-', lw=2)
#         else:
#             plt.plot([pr_bb[0], pr_bb[2], pr_bb[2], pr_bb[0], pr_bb[0], ],
#                      [pr_bb[1], pr_bb[1], pr_bb[3], pr_bb[3], pr_bb[1], ], 'r-', lw=2)

    if box is not None and resize:  
        width, height = im.width, im.height
        x1 = (box[0] - box[2] / 2.0) * width
        y1 = (box[1] - box[3] / 2.0) * height
        x2 = (box[0] + box[2] / 2.0) * width
        y2 = (box[1] + box[3] / 2.0) * height
    else:
        if len(box) > 4:
            box = force_square(box)
        x1, y1, x2, y2 = box
    if not noshow:
        plt.plot([x1, x2, x2, x1, x1], [y1, y1, y2, y2, y1], line, lw=2)

    return [x1, y1, x2, y2]    

In [120]:
hero = None
d = dict()
for vot_class in vot_classes:
    print(vot_class)
    d[vot_class] = list()
    
#     print(vot_class, end=",")
    
    image_paths = [osp.join(vot_path, vot_class, x) for x in [y for y in sorted(os.listdir(osp.join(vot_path, vot_class))) if y.endswith("jpg")]]
    for i, image_path in enumerate(image_paths):
        image = Image.open(image_path).convert('RGB')
        sized = image.resize((m.width, m.height))
        boxes = do_detect(m, sized, 0.4, 0.3, use_cuda)
#         for box in boxes:
#             plot_single(image, box, line="g-", resize=True)
        gt = force_square(read_gt(osp.join(vot_path, vot_class, "groundtruth.txt"), i))
#         plot_single(image, gt, line="r-")
        
        if hero is None:
            hero = best_iou(boxes, gt, image.width, image.height)
        else:
            hero = choose_best_by_emb(hero, boxes)
        
        res = plot_single(image, hero, 'b-', resize=True, noshow=True)
        score = iou(res, gt)
        print(float(score), end=",", file=open("vot16_results/{}.csv".format(vot_class), 'a+'))
        d[vot_class].append(score)
        print(",".join([str(float(x)) for x in res]), file=open("vot16_predictions/{}.csv".format(vot_class), 'a+'))
#         print(score)
              
#         plt.show()
#         plt.close()

soccer1
road
sphere
glove
nature
matrix
ball1
fish1
singer3
bmx
book
shaking
sheep
girl
gymnastics1
motocross1
fish2
fernando
traffic
butterfly
ball2
marching
pedestrian1
car1
rabbit
handball2
crossing
tunnel
bag
blanket
dinosaur
fish3
singer2
racing
pedestrian2
motocross2
gymnastics3
iceskater1
birds1
hand
bolt2
bolt1
godfather
octopus
soldier
graduate
birds2
wiper
leaves
gymnastics4
handball1
singer1
gymnastics2
tiger
helicopter
fish4
car2
iceskater2
soccer2
basketball


In [124]:
a = []
for i in d:
    print(i, np.mean(d[i]))
    a += [np.mean(d[i])]

octopus 0.18898798169296702
godfather 0.00849249229353183
motocross2 0.020253678180414215
leaves 0.0033865534169098357
soldier 0.017144504014140496
sheep 0.05429507582026927
nature 0.002413901310938235
fernando 0.012003958987970768
dinosaur 0.1213449415543227
pedestrian1 0.007691337710331735
glove 6.196747811656735e-05
fish1 0.007970529776649991
singer1 0.047449920441162904
gymnastics3 0.009052786260359142
girl 0.04087188624834806
crossing 0.2555933270621049
blanket 0.06648658764922474
pedestrian2 0.4165422571774428
iceskater1 0.5911458253576228
bmx 0.6174732090842294
iceskater2 0.08838617242631135
road 0.437765870367873
gymnastics4 0.026588860680933427
gymnastics1 0.14777306236226512
birds2 0.013404218125204573
book 0.006826739202419359
racing 0.13595079196024754
soccer1 0.051559112407648156
fish4 0.036746941949830185
fish2 0.007942625954621635
helicopter 0.05419932182412542
rabbit 0.07998759120050296
fish3 0.02775258388762703
birds1 0.025076047431887516
sphere 0.005874954002775605
ca

In [125]:
np.mean(a)

0.08777316776337443